In [5]:
#importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameWriter
from pyspark.sql.functions import monotonically_increasing_id
import os
import psycopg2

In [10]:
#set Java home
os.environ['JAVA_HOME'] = 'c:\Program Files\Java\jdk-17'

In [11]:
spark = SparkSession.builder \
    .appName("Nuga Bank ETL Pipeline") \
        .config("spark.jars", "postgresql-42.7.7.jar") \
        .getOrCreate()

In [12]:
spark